In [3]:
import json
import pandas as pd

In [4]:
with open('data.json') as json_file:
    data = json_file.read()

In [5]:
## read the data
school_data = json.loads(data)

In [6]:
## drop duplicates based on the location and time
df= pd.read_json('data.json', 'split' )
df= df.drop_duplicates(['lat','lng','time']).reset_index(drop= True)

In [7]:
df

,temperature,humidity,noise_level,wifiAccessPoints,lat,lng,accuracy,time
0,15,45,585,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538797,-79.666482,35,"1/20/2020, 7:51:10 AM"
1,19,18,582,"[{'macAddress': '20:3A:07:48:1A:04', 'ssid': '...",43.538823,-79.666474,33,"1/20/2020, 7:51:24 AM"
2,20,17,583,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538818,-79.666444,32,"1/20/2020, 7:51:33 AM"
3,20,17,663,"[{'macAddress': '88:75:56:C6:45:50', 'ssid': '...",43.538894,-79.666039,22,"1/20/2020, 7:52:50 AM"
4,20,18,668,"[{'macAddress': '88:75:56:87:6C:A4', 'ssid': '...",43.538940,-79.666016,20,"1/20/2020, 7:52:59 AM"
...,...,...,...,...,...,...,...,...
176,21,17,677,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538791,-79.666584,20,"1/20/2020, 2:49:43 PM"
177,20,18,678,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538801,-79.666574,20,"1/20/2020, 2:49:52 PM"
178,20,17,676,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538789,-79.666579,20,"1/20/2020, 2:50:00 PM"
179,20,18,676,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538802,-79.666590,20,"1/20/2020, 2:50:09 PM"


In [8]:
df['wifiAccessPoints'][0] # WifiAccesspoints include different SSID and SIgnal Strength in dictionary, which we extract next

[{'macAddress': '88:75:56:AF:20:10',
  'ssid': 'PDSB-ADMIN',
  'signalStrength': -58},
 {'macAddress': '88:75:56:AF:20:14',
  'ssid': 'PDSB-MEDIA',
  'signalStrength': -58},
 {'macAddress': '88:75:56:AF:20:11',
  'ssid': 'PDSB-WiFi',
  'signalStrength': -58},
 {'macAddress': '88:75:56:AF:20:15',
  'ssid': 'PDSB-GUEST',
  'signalStrength': -58},
 {'macAddress': '88:75:56:57:0C:94',
  'ssid': 'PDSB-MEDIA',
  'signalStrength': -89},
 {'macAddress': '88:75:56:57:0C:95',
  'ssid': 'PDSB-GUEST',
  'signalStrength': -87},
 {'macAddress': '88:75:56:87:6C:A4',
  'ssid': 'PDSB-MEDIA',
  'signalStrength': -83},
 {'macAddress': '20:3A:07:48:1A:00',
  'ssid': 'PDSB-ADMIN',
  'signalStrength': -55},
 {'macAddress': '20:3A:07:48:1A:04',
  'ssid': 'PDSB-MEDIA',
  'signalStrength': -54},
 {'macAddress': '88:75:56:87:6C:A0',
  'ssid': 'PDSB-ADMIN',
  'signalStrength': -82},
 {'macAddress': '20:3A:07:48:1A:05',
  'ssid': 'PDSB-GUEST',
  'signalStrength': -56},
 {'macAddress': '88:75:56:87:6C:A5',
  'ssid

Based on the observation of the WifiAccessPoints, we can see there are mainly four types of PDSB network:
  PDSB_Wifi - mainly for school students
  PDSB_GUEST - Mainly for guests and visitors to the schools
  PDSB_Media   Assume, it is mainly for downloading large media or streaming
  PDSB_Admin - for school admins and staffs
  
  Below, for each lat long, we extract the mean, minimum, and maximum signal strength for each of above four type Networks. we also extract the number of networks available in each category for each location. 

In [9]:
## drop duplicates based on the location and time

def get_ssid_features(ssid_mac_add, row_time):
    df_ssid= pd.DataFrame(ssid_mac_add)
    df_ssid.groupby('ssid').agg({'signalStrength': ['max'], 'macAddress': 'count'})
    df1= pd.merge(df_ssid, df_ssid.groupby('ssid').agg({'signalStrength': ['max', 'mean'], 'macAddress': 'count'}).reset_index(), on='ssid',how='outer')
    df1.rename(columns= {df1.columns[3]: 'max_signalStrength', df1.columns[4]: 'mean_signalStrength',
                        df1.columns[5]: 'count_mac'}, inplace= True)
    df1= df1.drop_duplicates(['ssid', 'max_signalStrength', 'mean_signalStrength',
                          'count_mac']).drop(['macAddress','signalStrength'], axis= 1).reset_index(drop=True)
    df1= pd.melt(df1, id_vars=['ssid'], value_vars=['max_signalStrength', 'mean_signalStrength','count_mac'])
    df1['variable_name'] = df1['ssid']+ '_' +df1['variable']
    df1= df1[['variable_name', 'value']]
    df1= df1.set_index('variable_name').T
    df1= df1.reset_index(drop= True)
    df1['time']= row_time
    return(df1)

In [10]:
df2= []
for row in range(0,df.shape[0]):
      df1= get_ssid_features(df['wifiAccessPoints'][row], df['time'][row])
      df2.append(df1)

appended_data = pd.concat(df2, sort=False)
appended_data

/Users/punitrathore/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/punitrathore/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


,PDSB-ADMIN_max_signalStrength,PDSB-MEDIA_max_signalStrength,PDSB-WiFi_max_signalStrength,PDSB-GUEST_max_signalStrength,PDSB-ADMIN_mean_signalStrength,PDSB-MEDIA_mean_signalStrength,PDSB-WiFi_mean_signalStrength,PDSB-GUEST_mean_signalStrength,PDSB-ADMIN_count_mac,PDSB-MEDIA_count_mac,...,TPGuest_count_mac,Hotspot2E86_max_signalStrength,HaloCam-2a0ecf_max_signalStrength,Hotspot2E86_mean_signalStrength,HaloCam-2a0ecf_mean_signalStrength,Hotspot2E86_count_mac,HaloCam-2a0ecf_count_mac,LG Stylo 2 Plus_9586_max_signalStrength,LG Stylo 2 Plus_9586_mean_signalStrength,LG Stylo 2 Plus_9586_count_mac
0,-55.0,-54.0,-56.0,-56.0,-69.750000,-74.666667,-69.750000,-74.666667,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-55.0,-53.0,-55.0,-55.0,-71.600000,-68.250000,-72.000000,-72.000000,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-55.0,-54.0,-54.0,-55.0,-71.800000,-72.000000,-72.400000,-70.250000,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-49.0,-54.0,-53.0,-46.0,-76.250000,-77.250000,-75.750000,-75.250000,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-63.0,-66.0,-67.0,-66.0,-69.500000,-71.500000,-71.500000,-70.000000,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-49.0,-52.0,-52.0,-50.0,-68.500000,-67.750000,-69.000000,-67.500000,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-54.0,-56.0,-52.0,-53.0,-65.333333,-65.666667,-64.333333,-65.000000,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-64.0,-64.0,-65.0,-62.0,-70.666667,-69.666667,-71.333333,-69.333333,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-67.0,-72.0,-72.0,-73.0,-75.750000,-79.600000,-78.500000,-78.500000,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
final_df= pd.merge(df, appended_data, on='time',how='outer')
final_df.columns

Index(['temperature', 'humidity', 'noise_level', 'wifiAccessPoints', 'lat',
       'lng', 'accuracy', 'time', 'PDSB-ADMIN_max_signalStrength',
       'PDSB-MEDIA_max_signalStrength', 'PDSB-WiFi_max_signalStrength',
       'PDSB-GUEST_max_signalStrength', 'PDSB-ADMIN_mean_signalStrength',
       'PDSB-MEDIA_mean_signalStrength', 'PDSB-WiFi_mean_signalStrength',
       'PDSB-GUEST_mean_signalStrength', 'PDSB-ADMIN_count_mac',
       'PDSB-MEDIA_count_mac', 'PDSB-WiFi_count_mac', 'PDSB-GUEST_count_mac',
       'NETGEAR68_max_signalStrength', 'NETGEAR68_mean_signalStrength',
       'NETGEAR68_count_mac', 'Wireless2_max_signalStrength',
       'TPGuest_max_signalStrength', 'Wireless2_mean_signalStrength',
       'TPGuest_mean_signalStrength', 'Wireless2_count_mac',
       'TPGuest_count_mac', 'Hotspot2E86_max_signalStrength',
       'HaloCam-2a0ecf_max_signalStrength', 'Hotspot2E86_mean_signalStrength',
       'HaloCam-2a0ecf_mean_signalStrength', 'Hotspot2E86_count_mac',
       'HaloCam-2a

Here, we see that there are also other network available from TPGUEST, NETGEAR, HALOCAM, HOTSPOT, LG STYLO. Since these columns have  NaN values for most of the locations (>95% locations), we do not consider these network. 

In [12]:
final_df = final_df.iloc[:,0:20]# other columns have more than 95% NaN values so ignored them
final_df

,temperature,humidity,noise_level,wifiAccessPoints,lat,lng,accuracy,time,PDSB-ADMIN_max_signalStrength,PDSB-MEDIA_max_signalStrength,PDSB-WiFi_max_signalStrength,PDSB-GUEST_max_signalStrength,PDSB-ADMIN_mean_signalStrength,PDSB-MEDIA_mean_signalStrength,PDSB-WiFi_mean_signalStrength,PDSB-GUEST_mean_signalStrength,PDSB-ADMIN_count_mac,PDSB-MEDIA_count_mac,PDSB-WiFi_count_mac,PDSB-GUEST_count_mac
0,15,45,585,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538797,-79.666482,35,"1/20/2020, 7:51:10 AM",-55.0,-54.0,-56.0,-56.0,-69.750000,-74.666667,-69.750000,-74.666667,4.0,6.0,4.0,6.0
1,19,18,582,"[{'macAddress': '20:3A:07:48:1A:04', 'ssid': '...",43.538823,-79.666474,33,"1/20/2020, 7:51:24 AM",-55.0,-53.0,-55.0,-55.0,-71.600000,-68.250000,-72.000000,-72.000000,5.0,4.0,5.0,5.0
2,20,17,583,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538818,-79.666444,32,"1/20/2020, 7:51:33 AM",-55.0,-54.0,-54.0,-55.0,-71.800000,-72.000000,-72.400000,-70.250000,5.0,5.0,5.0,4.0
3,20,17,663,"[{'macAddress': '88:75:56:C6:45:50', 'ssid': '...",43.538894,-79.666039,22,"1/20/2020, 7:52:50 AM",-49.0,-54.0,-53.0,-46.0,-76.250000,-77.250000,-75.750000,-75.250000,4.0,4.0,4.0,4.0
4,20,18,668,"[{'macAddress': '88:75:56:87:6C:A4', 'ssid': '...",43.538940,-79.666016,20,"1/20/2020, 7:52:59 AM",-63.0,-66.0,-67.0,-66.0,-69.500000,-71.500000,-71.500000,-70.000000,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,21,17,677,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538791,-79.666584,20,"1/20/2020, 2:49:43 PM",-49.0,-52.0,-52.0,-50.0,-68.500000,-67.750000,-69.000000,-67.500000,4.0,4.0,4.0,4.0
177,20,18,678,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538801,-79.666574,20,"1/20/2020, 2:49:52 PM",-54.0,-56.0,-52.0,-53.0,-65.333333,-65.666667,-64.333333,-65.000000,3.0,3.0,3.0,3.0
178,20,17,676,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538789,-79.666579,20,"1/20/2020, 2:50:00 PM",-64.0,-64.0,-65.0,-62.0,-70.666667,-69.666667,-71.333333,-69.333333,3.0,3.0,3.0,3.0
179,20,18,676,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538802,-79.666590,20,"1/20/2020, 2:50:09 PM",-67.0,-72.0,-72.0,-73.0,-75.750000,-79.600000,-78.500000,-78.500000,4.0,5.0,4.0,4.0


In [13]:
final_df.to_csv('data_for_hackathon')

Our next objective is to aggregate these features e.g. noise, temperature, humidity etc for each location. However, it is  not possible to aggregate it reliable using raw latitude and longitude values. We divide the given area (based on minimum and maximum lat and lng values) into square grids (cells) of 5meters by 5 meters area, and then compute the mean features (e.g. noise,temp, humidity etc) for each cell (or cell_id). 

## Below we divide the given locations into grids. The following block of code may require extra packages to be installed. If it doesnt work, we have saved the output of this block in a CSV file called "data_for_hackathon_cell_id.csv" (skip next 3 cells)

In [13]:
#  If you dont want to installed following packages, we have saved the output of this block in a CSV file called "data_for_hackathon_cell_id.csv" 

from matplotlib import pyplot as plt
import mplleaflet
import numpy as np
import geopandas
import osmnx as ox
from descartes import PolygonPatch
import random

import scipy.io
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union
import osrm
import time
import overpy
import glob
import pychoropleth

In [14]:
bounds = pychoropleth.bounds_to_polygon([-79.6690, 43.5386, -79.6658,43.5393]) ## School Area polygon from given data

In [15]:
data_with_cell_id= pychoropleth.add_cell_id(data, 5, bounds=bounds, longitude="lng", latitude="lat")[1]
data_with_cell_id('geometry', axis= 1, inplace= True)
data_with_cell_id.to_csv("data_for_hackathon_cell_id.csv")

## Read the data with cell_id (the last column is cell_ID which is (lat cell and lng cell number)

In [14]:
data_with_cell_id= pd.read_csv('data_for_hackathon_cell_id.csv', index_col=0) # divide locatins in 5 by 5 sq meter  grids
data_with_cell_id

,temperature,humidity,noise_level,wifiAccessPoints,lat,lng,accuracy,time,PDSB-ADMIN_max_signalStrength,PDSB-MEDIA_max_signalStrength,...,PDSB-GUEST_max_signalStrength,PDSB-ADMIN_mean_signalStrength,PDSB-MEDIA_mean_signalStrength,PDSB-WiFi_mean_signalStrength,PDSB-GUEST_mean_signalStrength,PDSB-ADMIN_count_mac,PDSB-MEDIA_count_mac,PDSB-WiFi_count_mac,PDSB-GUEST_count_mac,cell_id
0,15,45,585,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538797,-79.666482,35,"1/20/2020, 7:51:10 AM",-55.0,-54.0,...,-56.0,-69.750000,-74.666667,-69.750000,-74.666667,4.0,6.0,4.0,6.0,"(40, 5)"
1,19,18,582,"[{'macAddress': '20:3A:07:48:1A:04', 'ssid': '...",43.538823,-79.666474,33,"1/20/2020, 7:51:24 AM",-55.0,-53.0,...,-55.0,-71.600000,-68.250000,-72.000000,-72.000000,5.0,4.0,5.0,5.0,"(40, 5)"
2,20,17,583,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538818,-79.666444,32,"1/20/2020, 7:51:33 AM",-55.0,-54.0,...,-55.0,-71.800000,-72.000000,-72.400000,-70.250000,5.0,5.0,5.0,4.0,"(41, 5)"
3,20,17,663,"[{'macAddress': '88:75:56:C6:45:50', 'ssid': '...",43.538894,-79.666039,22,"1/20/2020, 7:52:50 AM",-49.0,-54.0,...,-46.0,-76.250000,-77.250000,-75.750000,-75.250000,4.0,4.0,4.0,4.0,"(47, 7)"
4,20,18,668,"[{'macAddress': '88:75:56:87:6C:A4', 'ssid': '...",43.538940,-79.666016,20,"1/20/2020, 7:52:59 AM",-63.0,-66.0,...,-66.0,-69.500000,-71.500000,-71.500000,-70.000000,2.0,2.0,2.0,2.0,"(48, 8)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,21,17,677,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538791,-79.666584,20,"1/20/2020, 2:49:43 PM",-49.0,-52.0,...,-50.0,-68.500000,-67.750000,-69.000000,-67.500000,4.0,4.0,4.0,4.0,"(39, 4)"
177,20,18,678,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538801,-79.666574,20,"1/20/2020, 2:49:52 PM",-54.0,-56.0,...,-53.0,-65.333333,-65.666667,-64.333333,-65.000000,3.0,3.0,3.0,3.0,"(39, 5)"
178,20,17,676,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538789,-79.666579,20,"1/20/2020, 2:50:00 PM",-64.0,-64.0,...,-62.0,-70.666667,-69.666667,-71.333333,-69.333333,3.0,3.0,3.0,3.0,"(39, 4)"
179,20,18,676,"[{'macAddress': '88:75:56:AF:20:10', 'ssid': '...",43.538802,-79.666590,20,"1/20/2020, 2:50:09 PM",-67.0,-72.0,...,-73.0,-75.750000,-79.600000,-78.500000,-78.500000,4.0,5.0,4.0,4.0,"(39, 5)"


In [15]:
##find the average lat and lon for each grid 
grid_lat_lon= data_with_cell_id.groupby(['cell_id']).agg({'lat': ['mean'], 'lng': ['mean']})
grid_lat_lon.reset_index(inplace = True)

In [28]:
#following figure shows the lat lng of given data with cell_id in popup at markers locations
import folium
school_map= folium.Map(location = [43.538884, -79.666676],  min_zoom=18, max_zoom=18) 
for index, row in grid_lat_lon.iterrows():
    folium.Marker([row[('lat', 'mean')], row[('lng', 'mean')]], popup=row['cell_id']).add_to(school_map)
school_map.save('school_map.html')
school_map

Below compute the temperatuere, humidity , noisy, wifi (guest as well) signal strength based on the different
day session. We divide a day into 8 hours group as following, and compute mean feature values.

In [17]:
b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']

def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    
    
data_with_cell_id['time'] = pd.to_datetime(df['time'], errors='coerce')
data_with_cell_id['hour']= data_with_cell_id.time.dt.hour
data_with_cell_id['day_session'] = data_with_cell_id['hour'].apply(f)


data_with_cell_id.groupby(['cell_id','day_session']).agg({'temperature': 'mean', 
                                              'humidity': 'mean',
                                              'noise_level': 'mean',
                                             'PDSB-GUEST_mean_signalStrength':'mean',
                                             'PDSB-WiFi_mean_signalStrength': 'mean'})

temperature  humidity  noise_level  \
cell_id  day_session                                       
(25, 4)  Noon                20.0      17.0   681.000000   
(25, 5)  Morning             20.0      17.0   665.000000   
         Noon                20.0      17.0   680.750000   
(26, 0)  Morning             19.0      17.0   671.333333   
(26, 10) Morning             20.0      17.0   665.000000   
...                           ...       ...          ...   
(50, 13) Morning             20.0      17.5   666.000000   
         Noon                20.5      16.5   674.000000   
(50, 14) Morning             20.0      17.0   665.000000   
(50, 9)  Morning             20.0      17.5   665.000000   
(9, 2)   Morning             19.0      18.0   667.000000   

                      PDSB-GUEST_mean_signalStrength  \
cell_id  day_session                                   
(25, 4)  Noon                             -80.000000   
(25, 5)  Morning                          -74.200000   
         Noon                             -74.887500   
(26, 0)  Morning                          -77.435714   
(26, 10) Morning                          -72.600000   
...                                              ...   
(50, 13) Morning                          -77.500000   
         Noon                             -75.666667   
(50, 14) Morning                          -74.000000   
(50, 9)  Morning                          -66.250000   
(9, 2)   Morning                          -68.375000   

                      PDSB-WiFi_mean_signalStrength  
cell_id  day_session                                 
(25, 4)  Noon                            -81.250000  
(25, 5)  Morning                         -71.250000  
         Noon                            -77.241667  
(26, 0)  Morning                         -76.638095  
(26, 10) Morning                         -75.833333  
...                                             ...  
(50, 13) Morning                         -76.000000  
         Noon                            -77.000000  
(50, 14) Morning                         -74.500000  
(50, 9)  Morning                         -67.250000  
(9, 2)   Morning                         -69.833333  

[90 rows x 5 columns]

Below, we show the average, minimum and maximum temperature for each grid. Locations can be filtered based on the given threshold. 

In [18]:
grid_temperature_stats= data_with_cell_id.groupby(['cell_id']).agg({'lat': ['mean'], 'lng': ['mean'], 'temperature': ['mean', 'min', 'max']})
grid_temperature_stats.sort_values(by=[('temperature', 'mean')])

lat        lng temperature        
               mean       mean        mean min max
cell_id                                           
(40, 5)   43.538807 -79.666496   17.666667  15  19
(31, 4)   43.538758 -79.667049   18.000000  18  18
(27, 2)   43.538708 -79.667316   18.400000  16  20
(33, 6)   43.538876 -79.666934   18.666667  18  19
(35, 6)   43.538853 -79.666808   18.750000  17  20
...             ...        ...         ...  ..  ..
(49, 10)  43.539029 -79.665955   21.000000  20  23
(45, 2)   43.538674 -79.666197   21.000000  21  21
(27, 0)   43.538583 -79.667337   21.000000  21  21
(29, 8)   43.538976 -79.667179   21.000000  21  21
(30, 9)   43.539016 -79.667102   21.000000  20  22

[61 rows x 5 columns]

Visualiasation of temperature for each locations. 

In [29]:
##Map values ot colormap based on range
def rgb(minimum, maximum, value):
    if value>maximum:
        value= maximum
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b



grid_temperature_stats.reset_index(inplace = True)

## min to max temperature--> blue to green to red

minima = grid_temperature_stats[('temperature',  'mean')].min() ##minimum temperature 
maxima=  grid_temperature_stats[('temperature',  'mean')].max() ##maximum temperature

school_temperature_map= folium.Map(location = [43.538884, -79.666676],  min_zoom=18, max_zoom=18) 
for index, row in grid_temperature_stats.iterrows():
    countcolormap= '#%02X%02X%02X' % rgb(minima,maxima, row[('temperature',  'mean')])
    folium.CircleMarker(
         [row[('lat', 'mean')], row[('lng', 'mean')]], 
         popup= row[('temperature',  'mean')],
         color = countcolormap,
         key_on = row[('temperature',  'mean')],
         fill= countcolormap,
         fill_opacity=0.7
         ).add_to(school_temperature_map)

school_temperature_map.save('school_temperature_map.html')
school_temperature_map

Below, we show the average, minimum and maximum humidity for each grid. Locations can be filtered based on the given threshold. 

In [20]:
grid_humidity_stats= data_with_cell_id.groupby(['cell_id']).agg({'lat': ['mean'], 'lng': ['mean'], 'humidity': ['mean', 'min', 'max']})
grid_humidity_stats.sort_values(by=[('humidity', 'mean')])

lat        lng   humidity        
               mean       mean       mean min max
cell_id                                          
(47, 6)   43.538872 -79.666067  16.000000  16  16
(45, 2)   43.538674 -79.666197  16.000000  16  16
(29, 8)   43.538976 -79.667179  16.000000  16  16
(27, 0)   43.538583 -79.667337  16.000000  16  16
(49, 12)  43.539126 -79.665926  16.500000  16  17
...             ...        ...        ...  ..  ..
(31, 9)   43.539015 -79.667052  18.500000  18  19
(30, 2)   43.538712 -79.667105  19.000000  19  19
(27, 2)   43.538708 -79.667316  19.200000  18  23
(31, 2)   43.538708 -79.667074  24.600000  18  48
(40, 5)   43.538807 -79.666496  26.333333  16  45

[61 rows x 5 columns]

In [21]:
grid_humidity_stats.reset_index(inplace = True)

## min to max temperature--> blue to green to red

minima = grid_humidity_stats[('humidity',  'mean')].min() ##minimum temperature 
maxima=  grid_humidity_stats[('humidity',  'mean')].max() ##maximum temperature

school_humidity_map= folium.Map(location = [43.538884, -79.666676],  min_zoom=18, max_zoom=18) 
for index, row in grid_humidity_stats.iterrows():
    countcolormap= '#%02X%02X%02X' % rgb(minima,maxima, row[('humidity',  'mean')])
    folium.CircleMarker(
         [row[('lat', 'mean')], row[('lng', 'mean')]], 
         popup= row[('humidity',  'mean')],
         color = countcolormap,
         key_on = row[('humidity',  'mean')],
         fill= countcolormap,
         fill_opacity=0.7
         ).add_to(school_humidity_map)
    
school_humidity_map.save('school_humidity_map.html')
school_humidity_map

Here we compute the noise level for each grids. From the data,  noise_level unit is not clear. Usually Noise level is measured in DBA. https://www.osha.gov/SLTC/noisehearingconservation/loud.html. Without any knowledge about noise unit, we provide grid wise noise level in ascending order. Depending on the acceptable level of noise, less noisy grids can be filtered using given threshold. 

In [22]:
grid_noise_level_stats= data_with_cell_id.groupby(['cell_id']).agg({'lat': ['mean'], 'lng': ['mean'], 'noise_level': ['mean', 'min' , 'max']})
grid_noise_level_stats.sort_values(by=[('noise_level', 'mean')])

lat        lng noise_level          
               mean       mean        mean  min  max
cell_id                                             
(41, 5)   43.538818 -79.666444  583.000000  583  583
(40, 5)   43.538807 -79.666496  614.666667  582  677
(27, 3)   43.538725 -79.667302  664.000000  664  664
(50, 14)  43.539229 -79.665870  665.000000  665  665
(38, 11)  43.539085 -79.666594  665.000000  665  665
...             ...        ...         ...  ...  ...
(31, 2)   43.538708 -79.667074  680.600000  675  684
(37, 5)   43.538806 -79.666665  681.000000  681  681
(25, 4)   43.538787 -79.667405  681.000000  681  681
(32, 3)   43.538724 -79.667020  682.000000  679  685
(30, 2)   43.538712 -79.667105  682.000000  682  682

[61 rows x 5 columns]

Below, we visualize the noise level in school based on the different noise values for each grid locations (mean lat lng of grid). The color mapping is done based on the minimum and maximum value of given feature (here, noise-level). 
the noise value can be seen with color, as well as pop up upon clicking on the location in the map. 

In [23]:
#visualize the noise level in school based on the different noise values
## from min to max  --> from blue to green to Red 
grid_noise_level_stats.reset_index(inplace = True)

minima = data_with_cell_id.noise_level.min() ##minimum noise level in data
maxima= data_with_cell_id.noise_level.max() ##maximum noise level in data

school_noise_map= folium.Map(location = [43.538884, -79.666676],  min_zoom=18, max_zoom=18) 
for index, row in grid_noise_level_stats.iterrows():
    countcolormap= '#%02X%02X%02X' % rgb(minima,maxima, row[('noise_level', 'mean')])
    folium.CircleMarker(
         [row[('lat', 'mean')], row[('lng', 'mean')]], 
         popup= row[('noise_level', 'mean')],
         color = countcolormap,
         key_on = row[('noise_level', 'mean')],
         fill= countcolormap,
         fill_opacity=0.7
         ).add_to(school_noise_map)
    
school_noise_map.save('school_noise_map.html')
school_noise_map

Below we can find the grids which have strong internet signal strength for Wifi Internet. We compute the average signal strength of Wifi spots, Admin wifi, Guess Wifi, and Media Wifi. We also compute the total number of SSIDS available for each grids.


Here are some signal categories (from:  https://eyesaas.com/wi-fi-signal-strength/)

    Signal        StrengthExpected Quality Required For
  
-30 dBm	Maximum signal strength,     you are probably standing right next to the access point.	

-50 dBm	Anything down to this level can be considered excellent signal strength.

-60 dBm	Good, reliable signal strength.	

-67 dBm	Reliable signal strength.	The minimum for any service depending on a reliable connection and signal strength, such as voice over Wi-Fi and non-HD video streaming.

-70 dBm	Not a strong signal.	Light browsing and email.

-80 dBm	Unreliable signal strength, will not suffice for most services.	Connecting to the network.

-90 dBm	The chances of even connecting are very low at this level.

Grids with good signal strength can be filtered using above knowledge. Here we also find the grids (locations) which have more number of Wifi network available.

In [24]:
#grid_wifi_mean_signal_stats= data_with_cell_id.groupby(['cell_id']).agg({'PDSB-WiFi_mean_signalStrength': ['mean'], 'PDSB-WiFi_count_mac': 'count'})
grid_wifi_mean_signal_stats= data_with_cell_id.groupby(['cell_id']).agg({'lat': ['mean'], 'lng': ['mean'], 'PDSB-WiFi_mean_signalStrength': ['mean'], 'PDSB-WiFi_count_mac': 'count'})
grid_wifi_mean_signal_stats.sort_values(by=[('PDSB-WiFi_count_mac', 'count')], ascending=False)

,lat,lng,PDSB-WiFi_mean_signalStrength,PDSB-WiFi_count_mac
,mean,mean,mean,count
cell_id,,,,
"(40, 4)",43.538788,-79.666515,-69.312500,12
"(39, 4)",43.538788,-79.666557,-69.000000,11
"(39, 5)",43.538803,-79.666572,-72.108333,10
"(38, 12)",43.539129,-79.666615,-73.687500,8
"(49, 9)",43.538994,-79.665957,-71.741667,6
...,...,...,...,...
"(32, 9)",43.539004,-79.667019,-80.000000,1
"(34, 6)",43.538879,-79.666888,-80.000000,1


Visualization of Wifi Signal Strength based on locations

weak to strong signal strengh --> blue to green to red

In [25]:
grid_wifi_mean_signal_stats.reset_index(inplace = True)

## weak to strong signal strengh --> blue to green to red

minima = grid_wifi_mean_signal_stats[('PDSB-WiFi_mean_signalStrength',  'mean')].min() ##minimum number of network available
maxima=  grid_wifi_mean_signal_stats[('PDSB-WiFi_mean_signalStrength',  'mean')].max() ##maximum number of network available

school_wifi_network_strength_map= folium.Map(location = [43.538884, -79.666676],  min_zoom=18, max_zoom=18) 
for index, row in grid_wifi_mean_signal_stats.iterrows():
    countcolormap= '#%02X%02X%02X' % rgb(minima,maxima, row[('PDSB-WiFi_mean_signalStrength',  'mean')])
    folium.CircleMarker(
         [row[('lat', 'mean')], row[('lng', 'mean')]], 
         popup= row[('PDSB-WiFi_mean_signalStrength',  'mean')],
         color = countcolormap,
         key_on = row[('PDSB-WiFi_mean_signalStrength',  'mean')],
         fill= countcolormap,
         fill_opacity=0.7
         ).add_to(school_wifi_network_strength_map)
school_wifi_network_strength_map

Visualization of Number of Wifi Network Avaailable based on locations

weak to strong signal strengh --> blue to green to red

In [26]:
minima = grid_wifi_mean_signal_stats[('PDSB-WiFi_count_mac', 'count')].min() ##minimum number of network available
maxima=  grid_wifi_mean_signal_stats[('PDSB-WiFi_count_mac', 'count')].max() ##maximum number of network available

school_wifi_network_count_map= folium.Map(location = [43.538884, -79.666676],  min_zoom=18, max_zoom=18) 
for index, row in grid_wifi_mean_signal_stats.iterrows():
    countcolormap= '#%02X%02X%02X' % rgb(minima,maxima, row[('PDSB-WiFi_count_mac', 'count')])
    folium.CircleMarker(
         [row[('lat', 'mean')], row[('lng', 'mean')]], 
         popup= row[('PDSB-WiFi_count_mac', 'count')],
         color = countcolormap,
         key_on = row[('PDSB-WiFi_count_mac', 'count')],
         fill= countcolormap,
         fill_opacity=0.7
         ).add_to(school_wifi_network_count_map)
school_wifi_network_count_map

Here we compute the GUEST_Network signal strength and number of guest network for each grids. This information can be used to idenfity locations where Guests (e.g. parents, visitors) can  connect to the strong internet. 

In [27]:
grid_Guest_PDSB_mean_signal_stats= data_with_cell_id.groupby(['cell_id']).agg({'PDSB-GUEST_max_signalStrength': ['mean'], 'PDSB-GUEST_count_mac': 'count'})
grid_Guest_PDSB_mean_signal_stats.sort_values(by=[('PDSB-GUEST_count_mac', 'count')], ascending=False)

,PDSB-GUEST_max_signalStrength,PDSB-GUEST_count_mac
,mean,count
cell_id,,
"(40, 4)",-63.416667,12
"(39, 4)",-57.727273,11
"(39, 5)",-63.000000,10
"(38, 12)",-72.875000,8
"(49, 9)",-55.166667,6
...,...,...
"(32, 9)",-74.000000,1
"(34, 6)",-72.000000,1


Using above approach, Grids with better signal strength for ADMIN Network and MEDIA Network can also be identified for Admin PDSB and MEDIA PDSB